In [2]:
# import image module 
from IPython.display import Image  
# get the image 
Image(url="quantization.png", width=600) 

Quantization is a technique that reduces the precision of numerical values by representing them with fewer bits, similar to rounding numbers. For example, a 32-bit floating-point number (F32), which uses 32 bits to store a value with high precision, can be quantized to an 8-bit integer (INT8), using only 8 bits. This reduces the storage size by a factor of four.  In machine learning, this is like representing a weight in a neural network with less precision.  Quantization is used to decrease model size, speed up computations, lower memory usage, and improve energy efficiency, making models more practical for deployment on devices with limited resources like mobile phones or embedded systems.

Half precision, also known as FP16 or float16, is a data type that uses 16 bits to represent a floating-point number, compared to the 32 bits used by single precision (FP32).  This reduces the storage size and can speed up computations, making it useful in machine learning for training and inference, though it offers less precision than FP32 and can sometimes lead to numerical instability if not used carefully.

Fine-tuning a quantized model can sometimes lead to a slight reduction in accuracy compared to fine-tuning the original, higher-precision model.  However, this accuracy drop is often small and acceptable, especially considering the significant gains in model size, speed, and efficiency that quantization provides, and techniques like quantization-aware training can further minimize this loss.

For quantized models, calibration becomes even more important because the quantization process itself can distort the predicted probabilities.  Calibration helps to correct these distortions, ensuring that the quantized model's confidence scores are still reliable and reflect the true likelihood of events, even after the reduction in precision.  This makes the quantized model's predictions more trustworthy and useful in decision-making.

Symmetric quantization maps floating-point values to integers around zero, ensuring the range of quantized values is symmetric about zero (e.g., -127 to +127), which is often simpler to implement. - Batch Normalization (Symmetric Unsigned int 8 quantization)

Symmetric unsigned int 8 quantization maps floating-point values to unsigned 8-bit integers (0-255) in a way that's symmetric around zero. This means that the range of floating-point values is scaled and shifted so that zero is represented by the middle value of the integer range (around 127), and positive and negative values are mapped to integers with equal ranges on either side. This simplifies the quantization process and is often used for quantizing weights in neural networks.

Asymmetric quantization, on the other hand, maps floating-point values to integers across a potentially shifted range, allowing for better representation of data that is not evenly distributed around zero, but it requires more complex scaling and offset calculations during quantization and dequantization.

Asymmetric unsigned int 8 quantization maps floating-point values to the full range of unsigned 8-bit integers (0-255) without necessarily centering around zero.  This means the mapping can be optimized for data that is not symmetrically distributed.  A floating-point range is scaled and shifted so that its minimum value corresponds to 0 and its maximum value corresponds to 255.  This approach can provide better resolution for data with a skewed distribution, but requires storing and applying scaling and offset parameters for both quantization and dequantization, making it slightly more complex than symmetric quantization.

The zero point, representing the integer value corresponding to a floating-point zero, is crucial in quantization.  In symmetric quantization, it's typically near the middle of the integer range (e.g., 0 for int8, ~127 for uint8), and the mapping of floating-point values is symmetric around this point.  Conversely, in asymmetric quantization, the zero point corresponds to the minimum floating-point value being quantized, and can be any value within the integer range (often 0), requiring both a scaling factor and an offset for accurate mapping, unlike symmetric quantization which often only needs a scaling factor.

In quantization, the scale is a crucial factor that determines the mapping between floating-point and integer values.  It acts like a scaling factor during both quantization (floating-point to integer conversion) and dequantization (integer back to floating-point), essentially controlling the granularity of the quantization process; a smaller scale allows for finer-grained quantization with potentially higher accuracy, while a larger scale results in coarser quantization with reduced storage size but possibly lower accuracy.

Post-training quantization (PTQ) is a quantization technique applied after a model has been fully trained in floating-point precision.  It involves converting the model's weights and activations to lower-precision integer representations (like int8) without any further training or fine-tuning, offering a straightforward way to reduce model size and improve inference speed, though it may sometimes result in a slight drop in accuracy.

*Post-training quantization (PTQ) involves these key steps:*

Model Selection: Choose the pre-trained floating-point model you want to quantize.

Calibration (Optional but Recommended): Provide a small representative dataset (calibration set) to the model.  This data is used to determine the appropriate scaling factors and zero-points for quantization, helping to minimize accuracy loss.  The model processes this data, and the ranges of activations are observed.

Quantization: Apply the chosen quantization scheme (e.g., symmetric or asymmetric) to the model's weights and, if applicable, activations. This involves mapping the floating-point values to the desired integer representation (e.g., int8).  The scaling factors and zero-points (or offsets) determined in the calibration step (or derived directly from the weight ranges if calibration data is not used) are used in this mapping.

Evaluation: Evaluate the quantized model's performance on a held-out test set to assess any accuracy degradation.

Deployment: Deploy the quantized model for inference, benefiting from its reduced size and increased speed.

Quantization-aware training (QAT) is a quantization technique where the model is trained while simulating the effects of quantization.  During training, the model's weights and activations are represented in floating-point, but the forward and backward passes of training incorporate the rounding and clipping operations characteristic of quantization, allowing the model to adapt and learn to compensate for the eventual reduction in precision, leading to significantly better accuracy compared to post-training quantization.

*Quantization-aware training (QAT) involves these steps:*

Start with a Trained Model: Begin with a fully trained floating-point model.

Simulate Quantization: During the training process, modify the forward pass to mimic the effects of quantization:

Quantize Weights: Round the weights to the target integer precision (e.g., int8) for the forward pass calculations. However, the weights are still stored and updated in floating-point.
Quantize Activations (if applicable): Similarly, quantize the activations to the target integer precision for the forward pass.
Dequantize: After the quantized forward pass, dequantize the weights and activations back to floating-point for the backward pass. This is crucial; the gradients are calculated and applied to the floating-point weights.
Continue Training: Continue training the model with this simulated quantization.  The model learns to adjust its weights to minimize the impact of the simulated quantization during the forward pass.

Fine-tuning (Sometimes):  After the QAT process, a final fine-tuning step might be performed to further refine the model's accuracy.

Quantize for Deployment: Once training is complete, the weights are truly quantized to the target integer format, and the model is ready for deployment.  Because the model was trained aware of the quantization, the accuracy loss is typically much lower than with post-training quantization.